In [1]:
import torch
import torch.nn as nn

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os

os.chdir('../')

In [3]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer,  TrainingArguments
from transformers import BertModel, BertConfig

import datasets
import evaluate

In [4]:
data = pd.read_csv('./data/data.csv')

In [5]:
data.head(1)

,antigen,TCR,interaction
0,AAGIGILTV,CAISEVGVGQPQHF,1


In [6]:
data.drop(columns=['interaction'], inplace=True)

In [7]:
# Apply a lambda function to insert spaces between characters
data['antigen'] = data['antigen'].apply(lambda x: ' '.join(list(x)))
data['TCR'] = data['TCR'].apply(lambda x: ' '.join(list(x)))

In [8]:
data

,antigen,TCR
0,A A G I G I L T V,C A I S E V G V G Q P Q H F
1,A A G I G I L T V,C A S S L S F G T E A F F
2,A A R A V F L A L,C A S L G A Q N N E Q F
3,A A R A V F L A L,C A S S Y S T G D E Q Y F
4,A I M D K N I I L,C A S S V D G G S Q P Q H F
...,...,...
130466,Y M G V S Y E M,C A S S P N H G G H S P L H F
130467,Y M G V S Y E M,C A S S P M G G H S P L H F
130468,F L G I Y T V T V V,C A S F L G G T G T E A F F
130469,F L G I Y T V T V V,C A S S A L R L S Q P G E Q Y F


In [9]:
# Convert the DataFrame to a Hugging Face dataset
dataset = datasets.Dataset.from_pandas(data)

In [10]:
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['antigen', 'TCR'],
        num_rows: 104376
    })
    test: Dataset({
        features: ['antigen', 'TCR'],
        num_rows: 26095
    })
})

In [11]:
dataset['train'].features


{'antigen': Value(dtype='string', id=None),
 'TCR': Value(dtype='string', id=None)}

In [12]:
from src.model import BERT_CONFIG

In [13]:
config = BERT_CONFIG

In [14]:
tokenizer = AutoTokenizer.from_pretrained("antigen", config=config)
tokenizer.model_max_length = 64


In [15]:
model = AutoModelForMaskedLM.from_config(BERT_CONFIG)

In [16]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(25, 512, padding_idx=0)
      (position_embeddings): Embedding(64, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-7): 8 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=

In [17]:
model.config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 64,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 25
}

In [18]:
column_names = list(dataset["train"].features)

In [19]:
column_names

['antigen', 'TCR']

[CLS]antigen[SEP]TCR[EOS]

In [20]:
# def tokenize_function(examples):
#     return tokenizer(examples[column_names[0]], examples[column_names[1]], return_special_tokens_mask=False,
#                      padding='longest', truncation='longest_first', return_tensors="pt")

def tokenize_function(examples):
    return tokenizer(examples[column_names[0]], examples[column_names[1]], return_special_tokens_mask=False,
                     padding='max_length', truncation=True, return_tensors="pt")

In [21]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on every text in dataset",
    )

Running tokenizer on every text in dataset:   0%|          | 0/104376 [00:00<?, ? examples/s]

Running tokenizer on every text in dataset:   0%|          | 0/26095 [00:00<?, ? examples/s]

In [22]:
# tokenized_datasets['train']
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 104376
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 26095
    })
})

In [23]:
print(tokenized_datasets['train'][0]['input_ids'])

[1, 21, 5, 12, 23, 9, 23, 9, 5, 14, 16, 2, 15, 12, 6, 24, 19, 23, 10, 23, 6, 6, 13, 20, 6, 17, 5, 24, 11, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
for i in range(10):
    print(f"i; {i}, input_ids: {len(tokenized_datasets['train']['input_ids'][i])}, attention_mask: {len(tokenized_datasets['train']['attention_mask'][i])}")

i; 0, input_ids: 64, attention_mask: 64
i; 1, input_ids: 64, attention_mask: 64
i; 2, input_ids: 64, attention_mask: 64
i; 3, input_ids: 64, attention_mask: 64
i; 4, input_ids: 64, attention_mask: 64
i; 5, input_ids: 64, attention_mask: 64
i; 6, input_ids: 64, attention_mask: 64
i; 7, input_ids: 64, attention_mask: 64
i; 8, input_ids: 64, attention_mask: 64
i; 9, input_ids: 64, attention_mask: 64


In [25]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

'[CLS] E L A G I G I L T V [SEP] C A S H R G Q G S S Y N S P L H F [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [26]:
train_dataset = tokenized_datasets["train"]

In [27]:
eval_dataset = tokenized_datasets["test"]

In [28]:
metric = evaluate.load("accuracy")

Using the latest cached version of the module from C:\Users\tolayi1\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Sun Dec  3 08:53:35 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [29]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by prepxrocess_logits_for_metrics
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

In [30]:
mlm_probability = 0.15 # Percentage of data to mask

In [31]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm_probability=mlm_probability)

In [32]:
training_args = TrainingArguments(output_dir='./trainBERT', evaluation_strategy="epoch",
                                  learning_rate=2e-5, per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16, num_train_epochs=1,
                                  weight_decay=0.01)

In [33]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [34]:
train_result = trainer.train()
trainer.save_model()  # Saves the tokenizer too for easy upload

  0%|          | 0/6524 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.0604, 'learning_rate': 1.846719803801349e-05, 'epoch': 0.08}
{'loss': 1.645, 'learning_rate': 1.6934396076026978e-05, 'epoch': 0.15}
{'loss': 1.3925, 'learning_rate': 1.540159411404047e-05, 'epoch': 0.23}
{'loss': 1.2647, 'learning_rate': 1.3868792152053956e-05, 'epoch': 0.31}
{'loss': 1.1819, 'learning_rate': 1.2335990190067444e-05, 'epoch': 0.38}
{'loss': 1.1285, 'learning_rate': 1.0803188228080932e-05, 'epoch': 0.46}
{'loss': 1.1055, 'learning_rate': 9.270386266094421e-06, 'epoch': 0.54}
{'loss': 1.068, 'learning_rate': 7.73758430410791e-06, 'epoch': 0.61}
{'loss': 1.0337, 'learning_rate': 6.2047823421213984e-06, 'epoch': 0.69}
{'loss': 1.0275, 'learning_rate': 4.6719803801348865e-06, 'epoch': 0.77}
{'loss': 0.9909, 'learning_rate': 3.1391784181483754e-06, 'epoch': 0.84}
{'loss': 1.001, 'learning_rate': 1.6063764561618639e-06, 'epoch': 0.92}
{'loss': 0.9866, 'learning_rate': 7.357449417535255e-08, 'epoch': 1.0}


  0%|          | 0/1631 [00:00<?, ?it/s]

{'eval_loss': 0.9536764621734619, 'eval_runtime': 22.6219, 'eval_samples_per_second': 1153.528, 'eval_steps_per_second': 72.098, 'epoch': 1.0}
{'train_runtime': 322.1116, 'train_samples_per_second': 324.037, 'train_steps_per_second': 20.254, 'train_loss': 1.2209553373583544, 'epoch': 1.0}


In [35]:
metrics = train_result.metrics
metrics["train_samples"] = len(train_dataset)
trainer.save_metrics("train", metrics)


In [36]:
metrics = trainer.evaluate()
metrics["eval_samples"] = len(eval_dataset)
trainer.save_metrics("eval", metrics)

  0%|          | 0/1631 [00:00<?, ?it/s]

In [ ]:
trainer.save_state()

In [ ]:
import math

In [ ]:
try:
    perplexity = math.exp(metrics["eval_loss"])
except OverflowError:
    perplexity = float("inf")
metrics["perplexity"] = perplexity

In [ ]:
trainer.save_metrics("eval", metrics)